In [1]:
# imports
import os,sys
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pandas as pd
import time

# import some helper functions
from proc_helpers import buffer_point_polygon_overlay,pp_summarize_ndvi_with_qa_dir

In [2]:
points_shp = '../../gis/moved_locations.shp'
extra_erase_shp = ['../../gis/VillplusGR.shp', '../../gis/Agin_Villages.shp']
df = gpd.read_file(points_shp)
print(df.crs)
print(gpd.read_file(extra_erase_shp[0]).crs)
print(gpd.read_file(extra_erase_shp[1]).crs)

# ensure the points are in the same CRS as polygons... for some reason they are different
df = df.to_crs(epsg='32736')

{'init': 'epsg:32636'}
{'init': 'epsg:32736'}
{'init': 'epsg:32736'}


In [3]:
df.crs == {'init':'epsg:32734'}
df.head()

,VillageNam,LocationID,NewName,NewID,geometry
0,Agincourt,BBBCE,Agincourt,01,POINT (323325.9884000002 7254571.9086)
1,Agincourt,BBBCN,Agincourt,01,POINT (323793.302 7254205.93)
2,Agincourt,BBBEH,Agincourt,01,POINT (323905.9172000003 7253881.413900001)
3,Agincourt,BBBEN,Agincourt,01,POINT (323416.6338000002 7253991.3725)
4,Agincourt,BBBJY,Agincourt,01,POINT (324492.3450999995 7252748.390000001)


In [ ]:
# buff_erase_df = create_buffer_point_polygon_overlay(df, method='difference', num_points=2)

# use this one for the points specified in the shapefile (_v2)
# buff_erase_df = create_buffer_point_polygon_overlay_v2(df, method='difference') 
# buff_erase_df.crs = {'init' :'epsg:32736'}

# use this one to erase a list of shapefiles from the buffered points. Those shapefiles must have same CRS
#buff_erase_df = create_buffer_point_polygon_overlay_v3(df, method='difference', num_points_fld='NUMHH', erase_shp_files=[extra_erase_shp]) 

# use this one if providing a data frame of points and to erase a list of shapefiles from buffered points.
test_df = df[df['NewID'].astype('int') < 3]
buff_erase_df = buffer_point_polygon_overlay(test_df, erase_shp_files=extra_erase_shp)

buff_erase_df.plot()

C:\Projects\south_africa\south_africa_landsat\jp\proc_helpers.py:341: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['geometry'] = df['geometry'].buffer(buff_dist)


{'init': 'epsg:32736'} {'init': 'epsg:32736', 'no_defs': True}


In [14]:
df.loc[df['NewID'].astype('int') < 3].shape

(2526, 5)

In [10]:
# this will show you how many geometries now exist for processing
df['NewID'].unique()

array(['01', '23', '02', '03', '04', '24', '25', '26', '27', '05', '12',
       '15', '06', '07', '11', '08', '19', '20', '21', '09', '10', '18',
       '31', '36', '29', '13', '14', '17', '30', '28', '16'], dtype=object)

## get the NDVI and pixel_qa raster paths and check that they match up
### specify the qa_dir and ndvi_dir with trailing '/' character or the functions will not run

In [6]:
qa_dir = '../landsat/test/qa/'
ndvi_dir = '../landsat/test/ndvi/'


In [7]:
t0 = time.time()
test_function_pp = pp_summarize_ndvi_with_qa_dir(ndvi_dir, qa_dir, buff_erase_df, 'median')
t1 = time.time()

## the number of cores and geometries will change...
#print('parallel across 6 cores and 28 geometries is: {}'.format(t1-t0))


## don't run this, it operates in serial and is slow
# t0 = time.time()
# test_function = summarize_ndvi_with_qa_dir(ndvi_dir, qa_dir, buff_erase_df, 'median')
# t1 = time.time()
# print('serial and 28 geometries is: {}'.format(t1-t0))

epsg_df: 32736, epsg_raster: 32636
epsg_df: {'init': 'epsg:32636', 'no_defs': True}, epsg_raster: 32636


C:\projects\RD\south_africa\jp\proc_helpers.py:534: UserWarning: Warning: converting a masked element to nan.
  ndvi_df = geo_df.join(pd.DataFrame(np.array(all_vals).T, columns=landsat_columns), how='outer')


In [ ]:
# change the output filename to your desire
test_function_pp.to_crs(epsg=32736).to_file('test_landsat_summaries_1997_2017_median_maskCloudWater.shp')

# everything below here is testing and doesn't need to run
## test number of points for scaling